## **MIMIC-III**: Pre-processing

This notebook contains the pre-processing of the [MIMIC-III](https://physionet.org/content/mimiciii/1.4/) dataset. 

In [1]:
from data import *
from utils import *
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

from data import *
from preprocessing import *


## 1. **Tabular data**

[MIMIC-IV Documentation](https://mimic.mit.edu/docs/iv/). 

**Tables that we use**

* `Admissions`
* `Patients`
* `Services`


**Tables that could be used:**
* `emar`
* `d_hcpcs.csv.gz`
* `hpcsevents`
* `labevents`
* `microbiologyevents`



### Preprocess admissions and patients

In [2]:
data = preprocess_data()


##

## 2. **Images data**


The mimic-cxr-2.0.0-metadata.csv.gz file contains useful meta-data derived from the original DICOM files in MIMIC-CXR. The columns are:

* **dicom_id** - An identifier for the DICOM file. The stem of each JPG image filename is equal to the dicom_id.
* **PerformedProcedureStepDescription** - The type of study performed ("CHEST (PA AND LAT)", "CHEST (PORTABLE AP)", etc).
* **ViewPosition** - The orientation in which the chest radiograph was taken ("AP", "PA", "LATERAL", etc).
* **Rows** - The height of the image in pixels.
* **Columns** - The width of the image in pixels.
* **StudyDate** - An anonymized date for the radiographic study. All images from the same study will have the same date and time. Dates are anonymized, but chronologically consistent for each patient. Intervals between two scans have not been modified during de-identification.
* **StudyTime** - The time of the study in hours, minutes, seconds, and fractional seconds. The time of the study was not modified during de-identification.
* **ProcedureCodeSequence_CodeMeaning** - The human readable description of the coded procedure (e.g. "CHEST (PA AND LAT)". Descriptions follow Simon-Leeming codes [11].
* **ViewCodeSequence_CodeMeaning** - The human readable description of the coded view orientation for the image (e.g. "postero-anterior", "antero-posterior", "lateral").
* **PatientOrientationCodeSequence_CodeMeaning** - The human readable description of the patient orientation during the image acquisition. Three values are possible: "Erect", "Recumbent", or a null value (missing).

In [ ]:
# Info from the images
info_jpg = pd.read_csv('../fake_data/mimic-cxr-2.0.0-metadata.csv')
info_jpg.head()

Note that "No Finding" is the absence of any of the 13 descriptive labels and a check that the text does not mention a specified set of other common findings beyond those covered by the descriptive labels. Each label column contains one of four values: 1.0, -1.0, 0.0, or missing. These labels have the following interpretation:

* **1.0** - The label was positively mentioned in the associated study, and is present in one or more of the corresponding images e.g. "A large pleural effusion"
* **0.0** - The label was negatively mentioned in the associated study, and therefore should not be present in any of the corresponding images e.g. "No pneumothorax."
* **-1.0** - The label was either: (1) Explicit uncertainty or (2) Ambiguous language
* **Missing** (empty element) - No mention of the label was made in the report

In [ ]:
labels_data = pd.read_csv('../fake_data/mimic-cxr-2.0.0-chexpert.csv')
labels_data.head()

In [ ]:
def list_images(base_path):
    """
    Recursively lists all image files starting from the base path.
    Assumes that images have extensions typical for image files (e.g., .jpg, .jpeg, .png).
    """
    image_files = []
    for subdir, dirs, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(subdir, file))
    return image_files

image_files = list_images('../fake_data/files')
image_files

In [ ]:
image_labels_mapping = create_image_labels_mapping(image_files, labels_data, info_jpg)
        
image_labels_mapping[image_files[0]]    

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image, ImageOps

class MedicalImagesDataset(Dataset):
    def __init__(self, data_dict, size=224, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.size = size
        self.classes = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 
                        'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 
                        'Lung Opacity', 'No Finding', 'Pleural Effusion', 
                        'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']
        # Organize paths by subject_id and study_id
        self.organized_paths = self._organize_paths()
        # Filter out pairs where both images are None
        self.organized_paths = {k: v for k, v in self.organized_paths.items() if v['PA'] is not None or v['Lateral'] is not None}

    def _organize_paths(self):
        organized = {}
        for path in self.data_dict.keys():
            parts = path.split(os.sep)
            subject_id = parts[-3][1:]
            study_id = parts[-2][1:]
            key = (subject_id, study_id)

            if key not in organized:
                organized[key] = {'PA': None, 'Lateral': None}

            view_position = self.data_dict[path]['ViewPosition']
            if view_position in ['PA', 'Lateral']:
                organized[key][view_position] = path

        return organized

    def __len__(self):
        return len(self.organized_paths)

    def _load_and_process_image(self, path):
        if path:
            image = Image.open(path).convert('RGB')
        else:
            # Create a blank (black) image if path is None
            image = Image.new('RGB', (self.size, self.size))

        image = image.resize((self.size, self.size))

        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)

        return image

    def __getitem__(self, idx):
        subject_study_pair = list(self.organized_paths.keys())[idx]
        pa_path = self.organized_paths[subject_study_pair]['PA']
        lateral_path = self.organized_paths[subject_study_pair]['Lateral']

        # Load and process PA and Lateral images
        pa_image = self._load_and_process_image(pa_path)
        lateral_image = self._load_and_process_image(lateral_path)

        # Use one of the available paths to get labels (assuming they are the same for both views)
        labels_path = pa_path if pa_path else lateral_path

        if not labels_path:
            # Skip this patient if both PA and Lateral images are missing
            return None

        labels = self.data_dict[labels_path]
        label_values = [labels[class_name] if not np.isnan(labels[class_name]) else 0 for class_name in self.classes]
        label_tensor = torch.tensor(label_values, dtype=torch.float32)

        return pa_image, lateral_image, label_tensor

# Transform definition (if needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MedicalImagesDataset(image_labels_mapping, transform=transform)
dataloaders = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

for pa_images, lateral_images, labels in dataloaders:
    print(pa_images.shape)
    print(lateral_images.shape)
    print(labels.shape)
    break


In [ ]:
# Iterate over batches of image,label pairs
for i, (images, labels, view) in enumerate(dataloader):
    print(images.shape)
    print(labels.shape)
    print(view)
    break